In [21]:
import aiohttp
import asyncio
import urllib3
from aiohttp import ClientSession
import nest_asyncio
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Apply nest_asyncio to allow asyncio in Jupyter
nest_asyncio.apply()

# Disable insecure request warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Basic configuration
gateway_ip = '192.168.0.109'
network_pin = '13579246'
headers = {
    'Authorization': f'Bearer {network_pin}',
    'Content-Type': 'application/json'
}

# Cache for storing device information
device_cache = None

async def get_all_devices(session):
    global device_cache
    if device_cache is not None:
        return device_cache
    
    url = f'http://{gateway_ip}/devices'
    try:
        async with session.get(url, headers=headers, ssl=False) as response:
            if response.status == 200:
                data = await response.json()
                if 'result' in data:
                    device_cache = data['result']
                    return device_cache
                else:
                    logger.error(f"Unexpected response format: {data}")
            else:
                logger.error(f"Error: Unable to get devices. Status code: {response.status}")
    except aiohttp.ClientError as e:
        logger.error(f"Network error when getting devices: {str(e)}")
    except Exception as e:
        logger.error(f"Unexpected error when getting devices: {str(e)}")
    return None

async def print_all_devices(session):
    devices = await get_all_devices(session)
    if devices:
        print("\nAll devices:")
        for device in devices:
            print(f"Name: {device['name']}, ID: {device['id']}, Type: {device['type']}")
    else:
        print("Failed to retrieve devices.")

def control_url(device_id):
    return f'http://{gateway_ip}/devices/{device_id}/commands'

async def send_control_command(session, device_id, attribute, value):
    data = {
        'attribute': attribute,
        'value': value
    }
    url = control_url(device_id)
    try:
        async with session.post(url, json=data, headers=headers, ssl=False) as response:
            print(f'Device {device_id} - {attribute}: Status {response.status}, Response {await response.text()}')
    except aiohttp.ClientError as e:
        logger.error(f'Error controlling device {device_id}: {str(e)}')

async def control_devices(session, commands):
    tasks = [send_control_command(session, cmd['device_id'], cmd['attribute'], cmd['value']) for cmd in commands]
    await asyncio.gather(*tasks)

async def create_scene(session, scene_name, state):
    devices = await get_all_devices(session)
    if not devices:
        logger.error("Failed to get devices. Cannot create scene.")
        return
    
    commands = []
    for device in devices:
        device_id = device['id']
        device_type = device['type']
        
        if device_type in ['switch', 'dimmer']:
            commands.append({'device_id': device_id, 'attribute': 'status', 'value': state})
        
        if device_type == 'dimmer' and state:
            commands.append({'device_id': device_id, 'attribute': 'dimLevel', 'value': 100})
    
    print(f"Executing scene: {scene_name}")
    await control_devices(session, commands)
    print(f"Scene {scene_name} execution completed")

async def main():
    async with ClientSession() as session:
        while True:
            print("\nPlease select an option:")
            print("0. Test Connection and Get All Devices")
            print("1. Turn All On")
            print("2. Turn All Off")
            print("3. Exit")
            
            choice = input("Enter your choice (0/1/2/3): ")
            
            if choice == '0':
                await print_all_devices(session)
            elif choice == '1':
                await create_scene(session, "Turn All On", True)
            elif choice == '2':
                await create_scene(session, "Turn All Off", False)
            elif choice == '3':
                print("Exiting program")
                break
            else:
                print("Invalid option, please try again")

# For running in Jupyter Notebook
asyncio.get_event_loop().run_until_complete(main())

# For running as a standalone script
# if __name__ == "__main__":
#     asyncio.run(main())


Please select an option:
0. Test Connection and Get All Devices
1. Turn All On
2. Turn All Off
3. Exit


ERROR:__main__:Error: Unable to get devices. Status code: 401
ERROR:__main__:Failed to get devices. Cannot create scene.



Please select an option:
0. Test Connection and Get All Devices
1. Turn All On
2. Turn All Off
3. Exit
Executing scene: Turn All On
Device 63724 - status: Status 200, Response {"success":true}
Device 44914 - status: Status 200, Response {"success":true}
Device 34033 - status: Status 200, Response {"success":true}
Device 34033 - dimLevel: Status 200, Response {"success":true}
Device 34394 - status: Status 200, Response {"success":true}
Device 60413 - dimLevel: Status 200, Response {"success":true}
Device 46266 - status: Status 200, Response {"success":true}
Device 62990 - status: Status 200, Response {"success":true}
Device 43433 - status: Status 200, Response {"success":true}
Device 55750 - status: Status 200, Response {"success":true}
Device 60413 - status: Status 200, Response {"success":true}
Device 55750 - dimLevel: Status 200, Response {"success":true}
Device 56347 - status: Status 200, Response {"success":true}
Device 35458 - status: Status 200, Response {"success":true}
Device 